# Database Preprocessing and Cleaning
<hr>
Now that we got the data somewhere that it is easily accessible, now we just need to clean it and get it to a nice workable state. Once we accomplish that, then we'll be able to lock it down and make sure it can't be overwritten.

In [39]:
import pandas as pd
from sqlalchemy import create_engine
from pandas.io import sql

# Database creds, don't share on public repo
hostname='95.217.156.58'
dbname='crimeStats'
uname='sandwich'
pwd='321#@!IdiotSandwich'
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=hostname, db=dbname, user=uname, pw=pwd))


# get database as df
df = pd.read_sql('SELECT * FROM crimeTableMain', con=engine)

In [40]:
df = df.reset_index()
del df['index']

In [41]:
df.isna().sum()

level_0                            0
arrest_key                         0
arrest_date                        0
pd_cd                            270
pd_desc                         8950
ky_cd                           8961
ofns_desc                       8943
law_code                         207
law_cat_cd                     17391
arrest_boro                        9
arrest_precinct                    0
jurisdiction_code                  9
age_group                         16
perp_sex                           0
perp_race                          0
x_coord_cd                         1
y_coord_cd                         2
latitude                          22
longitude                          6
:@computed_region_efsh_h5xi    23792
:@computed_region_f5dn_yrer     7769
:@computed_region_yeji_bk3q     7821
:@computed_region_92fq_4b7q     7816
:@computed_region_sbqj_enih     7965
dtype: int64

With the amount of nulls we have, we should establish which values we absolutely need based on the needs of our project. Since we would like to map crime and look at crime historically, "latitude", "longitude", and "arrest_date" are non negotiable. Thankfully, at most, we should only lose 28 rows at most.

In [42]:
df = df[df['latitude'].notna()]

In [43]:
df = df[df['longitude'].notna()]

while age range isn't something we initially considered, since there are only 16 null values out of over 5 million, it could be valuable to drop those rows so we have that as a cleanly populated atttibute.

In [44]:
df = df[df['age_group'].notna()]

The next thing that stands out is x and y coordinates. I am curious if these may be redundant because we already have lat and long. 

After looking into it, it seems the "x_coord_cd" and "y_coord_cd" are attributes of the State Plane Coordinate System, which is an almost completely outdated way to lookup locations(<a href="https://www.usgs.gov/faqs/what-state-plane-coordinate-system-can-gps-provide-coordinates-these-values?qt-news_science_products=0#qt-news_science_products">see here</a>). Since that is the case we can remove those two rows completely.

In [45]:
del df['x_coord_cd']
del df['y_coord_cd']

Now Looking at all those "@" columns that have a bunch of null values.
 - computed_region_efsh_h5xi = Zip Code
 - computed_region_f5dn_yrer = Community District
 - computed_region_yeji_bk3q = Borough Boundaries
 - computed_region_92fq_4b7q = City Council Districts
 - computed_region_sbqj_enih = Police Precincts
 
What should we do with all these values? Since they provide actionable information (especially the zip code), we should probably keep them. 

In fact, we may actually be able to recover some information using geocoding libraries. We can see if we can grab some zip codes. The rest, we'll leave null, but keep on hand for specialized use if we decide they're useful for something since their nulls won't impede our main interests.

***update:*** upon investigation, these don't appear to be zip codes where the crime occurs. This means using the lat and long to provide the remaining zip codes wouldn't be right. Moreso, there are a lot of values like "1" and "26001", that aren't valid zip codes. This leads me to believe that this column is no use to us, so we'll get rid of it.

In [46]:
del df[':@computed_region_efsh_h5xi']

In [47]:
df

,level_0,arrest_key,arrest_date,pd_cd,pd_desc,ky_cd,ofns_desc,law_code,law_cat_cd,arrest_boro,...,jurisdiction_code,age_group,perp_sex,perp_race,latitude,longitude,:@computed_region_f5dn_yrer,:@computed_region_yeji_bk3q,:@computed_region_92fq_4b7q,:@computed_region_sbqj_enih
1,1,192799737,2019-01-26T00:00:00.000,177,SEXUAL ABUSE,116,SEX CRIMES,PL 1306503,F,M,...,0,45-64,M,BLACK,40.800694,-73.94110928599997,7.0,4,36.0,16
2,2,193260691,2019-02-06T00:00:00.000,None,None,None,None,PL 2203400,F,M,...,0,25-44,M,UNKNOWN,40.757839,-73.99121211099998,12.0,4,10.0,8
3,3,149117452,2016-01-06T00:00:00.000,153,RAPE 3,104,RAPE,PL 1302503,F,K,...,0,25-44,M,BLACK,40.648650,-73.95033556299995,61.0,2,11.0,40
4,4,190049060,2018-11-15T00:00:00.000,157,RAPE 1,104,RAPE,PL 1303501,F,K,...,0,25-44,M,BLACK,40.674583,-73.93022154099998,16.0,2,49.0,49
5,5,24288194,2006-09-13T00:00:00.000,203,"TRESPASS 3, CRIMINAL",352,CRIMINAL TRESPASS,PL 140100E,M,K,...,2,45-64,M,BLACK,40.671254,-73.926713851,16.0,2,49.0,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5011995,5011995,24068627,2006-08-28T00:00:00.000,849,"NY STATE LAWS,UNCLASSIFIED VIOLATION",677,OTHER STATE LAWS,LOC000000V,V,Q,...,0,25-44,M,WHITE,40.747590,-73.88439699899999,65.0,3,21.0,73
5011996,5011996,24566897,2006-09-30T00:00:00.000,705,"FORGERY,ETC.-MISD.",358,OFFENSES INVOLVING FRAUD,PL 1657100,M,Q,...,0,25-44,M,BLACK HISPANIC,40.748281,-73.87788142999993,65.0,3,21.0,73
5011997,5011997,24578122,2006-10-01T00:00:00.000,904,"INTOXICATED DRIVING,ALCOHOL",119,INTOXICATED/IMPAIRED DRIVING,VTL11920F2,F,Q,...,0,25-44,M,WHITE HISPANIC,40.752037,-73.86907874399995,65.0,3,21.0,73
5011998,5011998,24172707,2006-09-06T00:00:00.000,503,"CONTROLLED SUBSTANCE,INTENT TO SELL 3",117,DANGEROUS DRUGS,PL 2201601,F,K,...,0,45-64,M,BLACK,40.650052,-73.964169327,60.0,2,11.0,43


Another two attributes that don't have a lot of null values which make it worth dropping the NA values are ARREST_BORO and JURISDICTION_CODE. We could figure out the arrest boro based on the lat and long of the arrest, but with there only being 9/5 million null rows, it's easier to just delete them and move on.

In [48]:
df = df[df['arrest_boro'].notna()]
df = df[df['jurisdiction_code'].notna()]

In [49]:
# pd_cd and ky_cd can be deleted because they're internal use codes

del df['pd_cd']
del df['ky_cd']

In [50]:
# blanks law codes means we have no idea what was committed
# ofns_desc is also blank, so these 200 values don't do much for us

df[df['law_code'].isna()]

,level_0,arrest_key,arrest_date,pd_desc,ofns_desc,law_code,law_cat_cd,arrest_boro,arrest_precinct,jurisdiction_code,age_group,perp_sex,perp_race,latitude,longitude,:@computed_region_f5dn_yrer,:@computed_region_yeji_bk3q,:@computed_region_92fq_4b7q,:@computed_region_sbqj_enih
28,28,188086111,2018-09-27T00:00:00.000,None,None,None,None,B,50,0,25-44,M,WHITE HISPANIC,40.910270,-73.89711662799994,48.0,5,40.0,33
61,61,188086113,2018-09-27T00:00:00.000,None,None,None,None,B,50,0,25-44,M,WHITE HISPANIC,40.910270,-73.89711662799994,48.0,5,40.0,33
73,73,188785833,2018-10-15T00:00:00.000,None,None,None,None,K,67,0,25-44,M,BLACK,40.651465,-73.95423641599997,61.0,2,11.0,40
99,99,188304699,2018-10-03T00:00:00.000,None,None,None,None,B,47,0,25-44,M,WHITE HISPANIC,40.872162,-73.86614123499999,29.0,5,22.0,30
307,307,190932018,2018-12-09T00:00:00.000,None,None,None,None,Q,115,0,18-24,M,WHITE HISPANIC,40.751099,-73.86064222299996,65.0,3,21.0,73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5002571,5002571,24036545,2006-08-27T00:00:00.000,None,None,None,None,K,75,0,<18,M,WHITE HISPANIC,40.675194,-73.87342954799993,45.0,2,25.0,47
5004838,5004838,24028439,2006-08-25T00:00:00.000,None,None,None,None,M,14,3,25-44,M,BLACK,40.753939,-73.99484365499995,12.0,4,10.0,8
5005185,5005185,24028439,2006-08-25T00:00:00.000,None,None,None,None,M,14,3,25-44,M,BLACK,40.753939,-73.99484365499995,12.0,4,10.0,8
5005370,5005370,24289053,2006-09-13T00:00:00.000,None,None,None,None,Q,113,0,18-24,M,BLACK,40.673395,-73.775671989,41.0,3,46.0,71


In [51]:
df = df[df['law_code'].notna()]

In [38]:
df.sort_values('law_cat_cd')

,index,arrest_key,arrest_date,pd_desc,ofns_desc,law_code,law_cat_cd,arrest_boro,arrest_precinct,jurisdiction_code,age_group,perp_sex,perp_race,latitude,longitude,:@computed_region_f5dn_yrer,:@computed_region_yeji_bk3q,:@computed_region_92fq_4b7q,:@computed_region_sbqj_enih
1386459,1386459,146740364,2015-10-07T00:00:00.000,PL 1552500,Q,114,0,18-24,M,WHITE,1009044,222000,40.77598847900003,16862.000000,39,72.0,"LARCENY,PETIT FROM OPEN AREAS,UNCLASSIFIED",341.0,PETIT LARCENY
2556334,2556334,86459472,2012-08-27T00:00:00.000,PL 1552500,B,52,0,18-24,F,WHITE HISPANIC,1009690,257590,40.87367103500002,11272.000000,48,33.0,"LARCENY,PETIT FROM OPEN AREAS,UNCLASSIFIED",341.0,PETIT LARCENY
2556333,2556333,86577141,2012-09-04T00:00:00.000,PL 1200001,Q,109,0,25-44,M,ASIAN / PACIFIC ISLANDER,1032084,216954,40.762043893000055,13832.000000,22,67.0,ASSAULT 3,344.0,ASSAULT 3 & RELATED OFFENSES
2556332,2556332,86351033,2012-08-20T00:00:00.000,PL 1654000,S,120,0,25-44,F,WHITE,965505,163695,40.61596975600002,10692.000000,4,74.0,"STOLEN PROPERTY 3,POSSESSION",232.0,POSSESSION OF STOLEN PROPERTY 5
2556331,2556331,86440473,2012-08-26T00:00:00.000,LOC000000V,K,81,0,25-44,M,BLACK,1004586,189203,40.68598015100008,18181.000000,69,52.0,"NY STATE LAWS,UNCLASSIFIED VIOLATION",677.0,OTHER STATE LAWS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5009427,5009427,24288812,2006-09-12T00:00:00.000,"US CODE,UNCLASSIFIED",F.C.A. P.I.N.O.S.,FOA9000049,None,M,10,0,18-24,M,WHITE HISPANIC,40.743690,-74.00708194899995,12.0,4,10.0,6
5009916,5009916,24534184,2006-09-28T00:00:00.000,"FUGITIVE,FROM OTHER STATES",F.C.A. P.I.N.O.S.,FOA9000016,None,K,70,0,18-24,M,BLACK,40.649371,-73.96087589799998,60.0,2,11.0,43
5010246,5010246,24288812,2006-09-12T00:00:00.000,"US CODE,UNCLASSIFIED",F.C.A. P.I.N.O.S.,FOA9000049,None,M,10,0,18-24,M,WHITE HISPANIC,40.743690,-74.00708194899995,12.0,4,10.0,6
5011844,5011844,24079868,2006-08-30T00:00:00.000,"FUGITIVE,FROM OTHER STATES",F.C.A. P.I.N.O.S.,FOA9000016,None,M,34,0,25-44,M,WHITE HISPANIC,40.851112,-73.93525591199995,47.0,4,39.0,22
